In [1]:
%run Runge_Adams_FDA.ipynb

In [2]:
mu, t, h = symbols(r'\mu, t, h', real=True)
u, u1 = symbols('u, u1', cls=Function)

In [3]:
eq = u(t).diff(t, 2) + mu*(u(t)**2 - 1)*u(t).diff(t) + u(t)
eq

\mu*(u(t)**2 - 1)*Derivative(u(t), t) + u(t) + Derivative(u(t), (t, 2))

In [4]:
init((u(t), u1(t)), t, h)
set_clip(8, 7, Rational(0, 1))

In [5]:
expand((T(u(t+h) - u(t-h)))/(2*h))

h**6*Derivative(u(t), (t, 7))/5040 + h**4*Derivative(u(t), (t, 5))/120 + h**2*Derivative(u(t), (t, 3))/6 + Derivative(u(t), t)

In [6]:
def CompactAdamsBashforth5(g, y):
#     return expand(-10*g(y)/3 - 22*g(y.subs(t, t+h))/3 + 8*g(y.subs(t, t+2*h))/3)
    return expand(-10*g(y)/3 - 22*g(y.subs(t, t+h))/3 + 8*g(y.subs(t, t+2*h))/3  + (-9*y + 9*y.subs(t, t+h))/h)

def CompactAdamsBashforth7(g, y):
    return expand(-87*g(y)/11 - 651*g(y.subs(t, t+h))/11 - 387*g(y.subs(t, t+2*h))/11\
        + 45*g(y.subs(t, t+3*h))/11) + (-28*y - 475*y.subs(t, t+h)/11 + 783*y.subs(t, t+2*h)/11)/h

In [7]:
def g(y):
    return Matrix([\
        y[1],\
        -mu*(y[0]**2 - 1)*y[1] - y[0],\
    ])

def f(y):
    return Matrix([\
        clip_n(y[1]),\
        clip_n(-mu*(y[0]**2 - 1)*y[1] - y[0]),\
    ])
    
s = (('u(t)', 'y[0]'), ('u1(t)', 'y[1]'), (r'\mu', 'mu'))
def replace(r):
    for a, b in s:
        r = r.replace(a, b)
    return r

In [8]:
start = time.time()

## Gauss–Legendre method

In [9]:
set_clip(5, 4, Rational(0, 1))
r = GaussLegendre2(f, Matrix([u(t), u1(t)]))

In [10]:
r

Matrix([
[                                u1(t)],
[-\mu*u(t)**2*u1(t) + \mu*u1(t) - u(t)]])

In [11]:
F1 = clip((T(u(t+h))-T(u(t)))/h  - T(r[0]))
prn(F1, mu)
# prnlatex(F1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


u_{tt}/2

h^2 =>


u_{ttt}/6

h^3 =>


u_{tttt}/24

In [12]:
F2 = clip((T(u1(t+h))-T(u1(t)))/h  - T(r[1]))
prn(F2, mu)
# prnlatex(F2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


u1_{tt}/2

h^2 =>


u1_{ttt}/6

h^3 =>


u1_{tttt}/24

In [13]:
f1 = NF(F1, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f1, mu)
# prnlatex(f1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


-\mu*u1*(u - 1)*(u + 1)/2 - u/2

h^2 =>


-\mu**2*u1*(u - 1)**2*(u + 1)**2/3 - \mu*u*(u**2 - 2*u1**2 - 1)/3 + u1/3

h^3 =>


-\mu**3*u1*(u - 1)**3*(u + 1)**3/4 - \mu**2*u*(u - 1)*(u + 1)*(3*u**2 - 16*u1**2 - 3)/12 + \mu*u1*(8*u**2 - u1**2 - 3)/6 + u/4

In [14]:
f2 = NF(F2, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f2, mu)
# prnlatex(f2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


\mu**2*u1*(u - 1)**2*(u + 1)**2/2 + \mu*u*(u**2 - 2*u1**2 - 1)/2 - u1/2

h^2 =>


\mu**3*u1*(u - 1)**3*(u + 1)**3/3 + \mu**2*u*(u - 1)*(u + 1)*(u**2 - 5*u1**2 - 1)/3 - \mu*u1*(10*u**2 - u1**2 - 4)/6 - u/3

h^3 =>


\mu**4*u1*(u - 1)**4*(u + 1)**4/4 + \mu**3*u*(u - 1)**2*(u + 1)**2*(3*u**2 - 26*u1**2 - 3)/12 - \mu**2*u1*(31*u**4 - 20*u**2*u1**2 - 40*u**2 + 4*u1**2 + 9)/12 - \mu*u*(4*u**2 - 8*u1**2 - 3)/6 + u1/4

In [15]:
replace(str(f2[2].collect(mu)))

'mu**3*(y[0]**6*y[1]/3 - y[0]**4*y[1] + y[0]**2*y[1] - y[1]/3) + mu**2*(y[0]**5/3 - 5*y[0]**3*y[1]**2/3 - 2*y[0]**3/3 + 5*y[0]*y[1]**2/3 + y[0]/3) + mu*(-5*y[0]**2*y[1]/3 + y[1]**3/6 + 2*y[1]/3) - y[0]/3'

## "original" Runge–Kutta method

In [16]:
set_clip(7, 6, Rational(0, 1))
r = RungeKutta4(f, Matrix([u(t), u1(t)]))

In [17]:
F1 = clip((T(u(t+h))-T(u(t)))/h  - T(r[0]))
prn(F1, mu)
# prnlatex(F1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


\mu*u1*(u - 1)*(u + 1)/2 + (u + u_{tt})/2

h^2 =>


...

\mu**3*u1*(u - 1)**3*(u + 1)**3/24 + \mu**2*u*(u - 1)*(u + 1)*(u**2 - 8*u1**2 - 1)/24 - \mu*u1*(4*u**2 - u1**2 - 1)/12 - (u - u_{tttt})/24

h^4 =>


\mu**3*u*u1**2*(u - 1)**2*(u + 1)**2/8 + \mu**2*u1*(8*u**4 - 9*u**2*u1**2 - 8*u**2 + 5*u1**2)/48 + \mu*u*(u**2 - 3*u1**2)/24 + u_{ttttt}/120

h^5 =>


-\mu**4*u*u1**2*(u - 1)**3*(u + 1)**3/48 - \mu**3*u1*(u - 1)*(u + 1)*(4*u**4 - 13*u**2*u1**2 - 4*u**2 + 5*u1**2)/96 - \mu**2*u*(u**4 - 9*u**2*u1**2 - u**2 + 2*u1**4 + 5*u1**2)/48 + \mu*u1*(5*u**2 - u1**2)/96 + u_{tttttt}/720

In [18]:
F2 = clip((T(u1(t+h))-T(u1(t)))/h  - T(r[1]))
prn(F2, mu)
# prnlatex(F2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


-\mu**2*u1*(u - 1)**2*(u + 1)**2/2 - \mu*u*(u**2 - 2*u1**2 - 1)/2 + (u1 + u1_{tt})/2

h^2 =>


...

-\mu**4*u1*(u - 1)**4*(u + 1)**4/24 - \mu**3*u*(u - 1)**2*(u + 1)**2*(u**2 - 22*u1**2 - 1)/24 + \mu**2*u1*(29*u**4 - 30*u**2*u1**2 - 32*u**2 + 14*u1**2 + 3)/24 + \mu*u*(4*u**2 - 11*u1**2 - 1)/12 - (u1 - u1_{tttt})/24

h^4 =>


-5*\mu**4*u*u1**2*(u - 1)**3*(u + 1)**3/12 - \mu**3*u1*(u - 1)*(u + 1)*(32*u**4 - 83*u**2*u1**2 - 32*u**2 + 27*u1**2)/48 - \mu**2*u*(12*u**4 - 105*u**2*u1**2 - 12*u**2 + 26*u1**4 + 61*u1**2)/48 + \mu*u1*(29*u**2 - 10*u1**2)/48 + u1_{ttttt}/120

h^5 =>


5*\mu**5*u*u1**2*(u - 1)**4*(u + 1)**4/48 + \mu**4*u1*(u - 1)**2*(u + 1)**2*(5*u**4 - 30*u**2*u1**2 - 5*u**2 + 9*u1**2)/24 + \mu**3*u*(10*u**6 - 209*u**4*u1**2 - 20*u**4 + 144*u**2*u1**4 + 298*u**2*u1**2 + 10*u**2 - 112*u1**4 - 89*u1**2)/96 - \mu**2*u1*(98*u**4 - 152*u**2*u1**2 - 62*u**2 + 9*u1**4 + 32*u1**2)/96 - \mu*u*(9*u**2 - 31*u1**2)/96 + u1_{tttttt}/720

In [19]:
f1 = NF(F1, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f1, mu)
# prnlatex(f1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

0

h^4 =>


\mu**4*u1*(u - 1)**4*(u + 1)**4/120 + \mu**3*u*(u - 1)**2*(u + 1)**2*(u**2 - 7*u1**2 - 1)/120 - \mu**2*u1*(6*u**4 - 5*u**2*u1**2 - 8*u**2 + u1**2 + 2)/80 - \mu*u*(3*u**2 - 7*u1**2 - 2)/120 + u1/120

h^5 =>


\mu**5*u1*(u - 1)**5*(u + 1)**5/144 + \mu**4*u*(u - 1)**3*(u + 1)**3*(u**2 - 7*u1**2 - 1)/144 - \mu**3*u1*(u - 1)*(u + 1)*(16*u**4 - 19*u**2*u1**2 - 24*u**2 - u1**2 + 8)/288 - \mu**2*u*(2*u**4 - 13*u**2*u1**2 - 5*u**2 + 9*u1**2 + 3)/144 + \mu*u1*(12*u**2 + u1**2 - 6)/288 + u/144

In [20]:
f2 = NF(F2, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f2, mu)
# prnlatex(f2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


0

h^2 =>


...

0

h^4 =>


-\mu**5*u1*(u - 1)**5*(u + 1)**5/120 - \mu**4*u*(u - 1)**3*(u + 1)**3*(u**2 - 2*u1**2 - 1)/120 + \mu**3*u1*(u - 1)*(u + 1)*(15*u**2*u1**2 - 8*u**2 - 7*u1**2 + 8)/240 - \mu**2*u*(2*u**4 - 25*u**2*u1**2 + 4*u**2 + 10*u1**4 + 5*u1**2 - 6)/240 + \mu*u1*(7*u**2 - 6*u1**2 + 6)/240 - u/120

h^5 =>


-\mu**6*u1*(u - 1)**6*(u + 1)**6/144 - \mu**5*u*(u - 1)**4*(u + 1)**4*(u**2 - 3*u1**2 - 1)/144 + \mu**4*u1*(u - 1)**2*(u + 1)**2*(u**4 + 14*u**2*u1**2 - 6*u**2 - 2*u1**2 + 5)/144 - \mu**3*u*(2*u**6 - 47*u**4*u1**2 + 4*u**4 + 32*u**2*u1**4 + 34*u**2*u1**2 - 14*u**2 - 24*u1**4 + 13*u1**2 + 8)/288 + \mu**2*u1*(6*u**4 - 8*u**2*u1**2 + 6*u**2 + u1**4 + 2*u1**2 - 4)/96 + \mu*u*(u1**2 + 2)/96 + u1/144

In [21]:
replace(str(f2[4].collect(mu)))

'mu**5*(-y[0]**10*y[1]/120 + y[0]**8*y[1]/24 - y[0]**6*y[1]/12 + y[0]**4*y[1]/12 - y[0]**2*y[1]/24 + y[1]/120) + mu**4*(-y[0]**9/120 + y[0]**7*y[1]**2/60 + y[0]**7/30 - y[0]**5*y[1]**2/20 - y[0]**5/20 + y[0]**3*y[1]**2/20 + y[0]**3/30 - y[0]*y[1]**2/60 - y[0]/120) + mu**3*(y[0]**4*y[1]**3/16 - y[0]**4*y[1]/30 - 11*y[0]**2*y[1]**3/120 + y[0]**2*y[1]/15 + 7*y[1]**3/240 - y[1]/30) + mu**2*(-y[0]**5/120 + 5*y[0]**3*y[1]**2/48 - y[0]**3/60 - y[0]*y[1]**4/24 - y[0]*y[1]**2/48 + y[0]/40) + mu*(7*y[0]**2*y[1]/240 - y[1]**3/40 + y[1]/40) - y[0]/120'

## Dormand–Prince method 5 order

In [22]:
set_clip(8, 7, Rational(0, 1))
r = DormandPrince5(f, Matrix([u(t), u1(t)]))

In [23]:
F1 = clip((T(u(t+h))-T(u(t)))/h  - T(r[0]))
prn(F1, mu)
# prnlatex(F1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


\mu*u1*(u - 1)*(u + 1)/2 + (u + u_{tt})/2

h^2 =>


...

-\mu**4*u1*(u - 1)**4*(u + 1)**4/120 - \mu**3*u*(u - 1)**2*(u + 1)**2*(u**2 - 22*u1**2 - 1)/120 + \mu**2*u1*(29*u**4 - 30*u**2*u1**2 - 32*u**2 + 14*u1**2 + 3)/120 + \mu*u*(4*u**2 - 11*u1**2 - 1)/60 - (u1 - u_{ttttt})/120

h^5 =>


\mu**5*u1*(u - 1)**5*(u + 1)**5/600 + \mu**4*u*(u - 1)**3*(u + 1)**3*(18*u**2 - 805*u1**2 - 18)/10800 - \mu**3*u1*(u - 1)*(u + 1)*(7392*u**4 - 18265*u**2*u1**2 - 7824*u**2 + 5781*u1**2 + 432)/64800 - \mu**2*u*(1335*u**4 - 11429*u**2*u1**2 - 1497*u**2 + 2710*u1**4 + 6879*u1**2 + 162)/32400 + \mu*u1*(2119*u**2 - 662*u1**2 - 108)/21600 + (6*u + 5*u_{tttttt})/3600

h^6 =>


7589*\mu**5*u*u1**2*(u - 1)**4*(u + 1)**4/324000 + \mu**4*u1*(u - 1)**2*(u + 1)**2*(115326*u**4 - 583403*u**2*u1**2 - 115326*u**2 + 141093*u1**2)/2916000 + \mu**3*u*(94050*u**6 - 1877597*u**4*u1**2 - 188100*u**4 + 1221492*u**2*u1**4 + 2647382*u**2*u1**2 + 94050*u**2 - 901524*u1**4 - 769785*u1**2)/5832000 - \mu**2*u1*(207253*u**4 - 302289*u**2*u1**2 - 132123*u**2 + 17567*u1**4 + 62391*u1**2)/1458000 - \mu*u*(25871*u**2 - 73630*u1**2)/1944000 + u_{ttttttt}/5040

In [24]:
F2 = clip((T(u1(t+h))-T(u1(t)))/h  - T(r[1]))
prn(F2, mu)
# prnlatex(F2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


-\mu**2*u1*(u - 1)**2*(u + 1)**2/2 - \mu*u*(u**2 - 2*u1**2 - 1)/2 + (u1 + u1_{tt})/2

h^2 =>


...

\mu**5*u1*(u - 1)**5*(u + 1)**5/120 + \mu**4*u*(u - 1)**3*(u + 1)**3*(u**2 - 52*u1**2 - 1)/120 - \mu**3*u1*(u - 1)*(u + 1)*(20*u**4 - 50*u**2*u1**2 - 21*u**2 + 16*u1**2 + 1)/30 - \mu**2*u*(29*u**4 - 250*u**2*u1**2 - 32*u**2 + 60*u1**4 + 150*u1**2 + 3)/120 + \mu*u1*(69*u**2 - 22*u1**2 - 3)/120 + (u + u1_{ttttt})/120

h^5 =>


-\mu**6*u1*(u - 1)**6*(u + 1)**6/600 - \mu**5*u*(u - 1)**4*(u + 1)**4*(18*u**2 - 1729*u1**2 - 18)/10800 + \mu**4*u1*(u - 1)**2*(u + 1)**2*(4377*u**4 - 21707*u**2*u1**2 - 4512*u**2 + 5484*u1**2 + 135)/16200 + \mu**3*u*(7242*u**6 - 142305*u**4*u1**2 - 14916*u**4 + 93452*u**2*u1**4 + 202662*u**2*u1**2 + 8106*u**2 - 68972*u1**4 - 60357*u1**2 - 432)/64800 - \mu**2*u1*(32165*u**4 - 47472*u**2*u1**2 - 21189*u**2 + 2698*u1**4 + 9684*u1**2 + 324)/32400 - \mu*u*(2075*u**2 - 6130*u1**2 - 108)/21600 + (6*u1 + 5*u1_{tttttt})/3600

h^6 =>


-119239*\mu**6*u*u1**2*(u - 1)**5*(u + 1)**5/3564000 - \mu**5*u1*(u - 1)**3*(u + 1)**3*(7221384*u**4 - 88553803*u**2*u1**2 - 7221384*u**2 + 20501331*u1**2)/128304000 - \mu**4*u*(u - 1)*(u + 1)*(2928780*u**6 - 171517871*u**4*u1**2 - 5857560*u**4 + 234474782*u**2*u1**4 + 227007062*u**2*u1**2 + 2928780*u**2 - 148281182*u1**4 - 55489191*u1**2)/128304000 + \mu**3*u1*(100579861*u**6 - 358978597*u**4*u1**2 - 144206986*u**4 + 83749440*u**2*u1**4 + 323836246*u**2*u1**2 + 43627125*u**2 - 27203616*u1**4 - 31544433*u1**2)/128304000 + \mu**2*u*(16294737*u**4 - 150285356*u**2*u1**2 - 9874785*u**2 + 58326592*u1**4 + 44616750*u1**2)/128304000 - \mu*u1*(4582115*u**2 - 1493508*u1**2)/42768000 + u1_{ttttttt}/5040

In [25]:
f1 = NF(F1, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f1, mu)
# prnlatex(f1, mu)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

0

h^5 =>


\mu**5*u1*(u - 1)**5*(u + 1)**5/3600 + \mu**4*u*(u - 1)**3*(u + 1)**3*(3*u**2 - 25*u1**2 - 3)/10800 - \mu**3*u1*(u - 1)*(u + 1)*(192*u**4 - 265*u**2*u1**2 - 264*u**2 + 21*u1**2 + 72)/64800 - \mu**2*u*(30*u**4 - 179*u**2*u1**2 - 57*u**2 + 10*u1**4 + 129*u1**2 + 27)/32400 + \mu*u1*(49*u**2 - 2*u1**2 - 18)/21600 + u/3600

h^6 =>


\mu**6*u1*(u - 1)**6*(u + 1)**6/2100 + \mu**5*u*(u - 1)**4*(u + 1)**4*(270*u**2 - 2143*u1**2 - 270)/567000 - \mu**4*u1*(u - 1)**2*(u + 1)**2*(198216*u**4 - 243353*u**2*u1**2 - 295416*u**2 + 60633*u1**2 + 97200)/40824000 - \mu**3*u*(63360*u**6 - 526741*u**4*u1**2 - 204480*u**4 + 186036*u**2*u1**4 + 1017766*u**2*u1**2 + 218880*u**2 - 68892*u1**4 - 491025*u1**2 - 77760)/40824000 + \mu**2*u1*(368711*u**4 - 428448*u**2*u1**2 - 506211*u**2 - 836*u1**4 + 66912*u1**2 + 116640)/40824000 + \mu*u*(22213*u**2 - 71300*u1**2 - 19440)/13608000 - u1/2100

In [26]:
f2 = NF(F2, [u(t).diff(t), u1(t).diff(t)], [F1, F2], head=False)
prn(f2, mu)
# prnlatex(f2, mu)

h^0 =>


\mu*u1*(u - 1)*(u + 1) + u + u1_{t}

h^1 =>


0

h^2 =>


...

0

h^5 =>


-\mu**6*u1*(u - 1)**6*(u + 1)**6/3600 - \mu**5*u*(u - 1)**4*(u + 1)**4*(3*u**2 - 19*u1**2 - 3)/10800 + \mu**4*u1*(u - 1)**2*(u + 1)**2*(69*u**4 - 34*u**2*u1**2 - 114*u**2 - 12*u1**2 + 45)/32400 + \mu**3*u*(u - 1)*(u + 1)*(42*u**4 - 105*u**2*u1**2 - 114*u**2 - 148*u1**4 + 237*u1**2 + 72)/64800 - \mu**2*u1*(35*u**4 + 48*u**2*u1**2 - 129*u**2 - 2*u1**4 - 36*u1**2 + 54)/32400 - \mu*u*(5*u**2 - 10*u1**2 - 18)/21600 + u1/3600

h^6 =>


-\mu**7*u1*(u - 1)**7*(u + 1)**7/2100 - \mu**6*u*(u - 1)**5*(u + 1)**5*(1485*u**2 - 58459*u1**2 - 1485)/3118500 + \mu**5*u1*(u - 1)**3*(u + 1)**3*(31671792*u**4 - 89385119*u**2*u1**2 - 34237872*u**2 + 7372623*u1**2 + 2566080)/898128000 + \mu**4*u*(u - 1)*(u + 1)*(15263280*u**6 - 167930113*u**4*u1**2 - 32664960*u**4 + 132069466*u**2*u1**4 + 234275146*u**2*u1**2 + 19540080*u**2 - 22394986*u1**4 - 66345033*u1**2 - 2138400)/898128000 - \mu**3*u1*(92704853*u**6 - 204452321*u**4*u1**2 - 167926418*u**4 + 20231880*u**2*u1**4 + 166277198*u**2*u1**2 + 79498365*u**2 + 1299672*u1**4 - 13492269*u1**2 - 4276800)/898128000 - \mu**2*u*(14176491*u**4 - 99271528*u**2*u1**2 - 17330235*u**2 + 20284856*u1**4 + 47896230*u1**2 + 2566080)/898128000 + \mu*u1*(5264035*u**2 - 1746204*u1**2 - 570240)/299376000 + u/2100

In [27]:
replace(str(f2[5].collect(mu)))

'mu**6*(-y[0]**12*y[1]/3600 + y[0]**10*y[1]/600 - y[0]**8*y[1]/240 + y[0]**6*y[1]/180 - y[0]**4*y[1]/240 + y[0]**2*y[1]/600 - y[1]/3600) + mu**5*(-y[0]**11/3600 + 19*y[0]**9*y[1]**2/10800 + y[0]**9/720 - 19*y[0]**7*y[1]**2/2700 - y[0]**7/360 + 19*y[0]**5*y[1]**2/1800 + y[0]**5/360 - 19*y[0]**3*y[1]**2/2700 - y[0]**3/720 + 19*y[0]*y[1]**2/10800 + y[0]/3600) + mu**4*(23*y[0]**8*y[1]/10800 - 17*y[0]**6*y[1]**3/16200 - 7*y[0]**6*y[1]/900 + 7*y[0]**4*y[1]**3/4050 + 19*y[0]**4*y[1]/1800 - y[0]**2*y[1]**3/3240 - 17*y[0]**2*y[1]/2700 - y[1]**3/2700 + y[1]/720) + mu**3*(7*y[0]**7/10800 - 7*y[0]**5*y[1]**2/4320 - 13*y[0]**5/5400 - 37*y[0]**3*y[1]**4/16200 + 19*y[0]**3*y[1]**2/3600 + 31*y[0]**3/10800 + 37*y[0]*y[1]**4/16200 - 79*y[0]*y[1]**2/21600 - y[0]/900) + mu**2*(-7*y[0]**4*y[1]/6480 - y[0]**2*y[1]**3/675 + 43*y[0]**2*y[1]/10800 + y[1]**5/16200 + y[1]**3/900 - y[1]/600) + mu*(-y[0]**3/4320 + y[0]*y[1]**2/2160 + y[0]/1200) + y[1]/3600'